# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
# A quick view on the data
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


# Part II. Apache Cassandra coding portion 

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS song_events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('song_events')
except Exception as e:
    print(e)

## Create 3 tables to answer the following 3 queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1 Description
1. The query required artist, song title and song's length with condition on sessionId and ItemInSession
2. Based ont his info, we know that we need to have sessionId and ItemInSession as the primary key in order for the where statement to work and uniquely identify the row

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
dropquery = "DROP TABLE IF EXISTS music_history_by_sessionId_itemInSession "
query = "CREATE TABLE IF NOT EXISTS music_history_by_sessionId_itemInSession "
query = query + "(sessionId int,itemInSession int, artist text, title text,length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(dropquery)
    session.execute(query)
except Exception as e:
    print(e)


In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_history_by_sessionId_itemInSession (sessionId,itemInSession,artist, title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]),line[0], line[9], float(line[5])))

In [12]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist,title,length from music_history_by_sessionId_itemInSession where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Faithless', title='Music Matters (Mark Knight Dub)', length=495.30731201171875)


#### Query 2 Description
1. The query required artist name, song and user first and last name with condition on userId and sessionId and order by itemInSession
2. To uniquely identify the row, we need userId and SessionID. In this query, we create a compsite key using combination of (userId, sessionId) so that data with same userId and SessionId are co-located in the Cassandra
3. To sort the data, we added itemInSession as clustered Column

In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

dropquery = "DROP TABLE IF EXISTS user_history_by_userId_sessionId "
query = "CREATE TABLE IF NOT EXISTS user_history_by_userId_sessionId "
query = query + "(userId int,sessionId int,itemInSession int, artist text, title text,firstName text,lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(dropquery)
    session.execute(query)
except Exception as e:
    print(e)

                    

In [14]:
#Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_history_by_userId_sessionId (userId ,sessionId ,itemInSession ,artist , title ,firstName ,lastName )"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),line[0], line[9],line[1],line[4]))

In [15]:
#Query Data
query = "select artist,title,itemInSession from user_history_by_userId_sessionId where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(artist='Down To The Bone', title="Keep On Keepin' On", iteminsession=0)
Row(artist='Three Drives', title='Greece 2000', iteminsession=1)
Row(artist='Sebastien Tellier', title='Kilometer', iteminsession=2)
Row(artist='Lonnie Gordon', title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3)


#### Query 3 Description
1. The query requires firstName and last name of each user with where statement on song name
2. To uniquely identify the row, we need song name and first name as primary key

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

dropquery = "DROP TABLE IF EXISTS users_by_title "
query = "CREATE TABLE IF NOT EXISTS users_by_title "
query = query + "(title text,userId int, firstName text,lastName text, PRIMARY KEY (title,userId))"
try:
    session.execute(dropquery)
    session.execute(query)
except Exception as e:
    print(e)
            

In [17]:
#Insert Data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_title (title,userId ,firstName ,lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [18]:
#Query Data
query = "select title,firstName,lastName from users_by_title where title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(title='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(title='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(title='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [20]:
dropquery1 = "DROP TABLE IF EXISTS music_history_by_sessionId_itemInSession  "
dropquery2 = "DROP TABLE IF EXISTS user_history_by_userId_sessionId "
dropquery3 = "DROP TABLE IF EXISTS users_by_title "

try:
    session.execute(dropquery1)
    session.execute(dropquery2)
    session.execute(dropquery3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()

#### THE END